In [89]:
from Engine.Tools import phiFunction, relativeAngle, distanceCalc
from itertools import product
import numpy as np
import random

In [90]:
# Simulation setup
random.seed(123)
N: int = 100                                                           # Simulation size
Lag: int = 1                                                            # Lag value in the model
Phi: float = 50.0
Rho: float = 0.75
timeInterval: int = 1
meanPol: float = 25.0

In [91]:
# Geographical setup
GridSize: list = [x for x in range(-1, 2)]                                           # Size of the geographical grid for the simulation
Distance: float = 10.0                                                               # in Km
Location: dict = {i: x for i, x in enumerate(list(product(GridSize, repeat=2)))}     # Dictionary of coordinates for each integer-valued intersection point in the grid

In [92]:
# Environmental Setup
windSpeed: np.ndarray = np.clip(np.random.normal(loc=30.0, scale=10.0, size=[N]), 1.0, np.inf)               # Random non-negative wind speed
# windDirection: np.ndarray = np.degrees(np.arccos(np.cos(np.linspace(0, 2 * np.pi, N, endpoint=False))))    # Sequential wind direction 
windDirection: np.ndarray = np.random.uniform(low=0, high=360, size=[N])                                     # Random wind direction 

In [93]:
# Simulating variables
K: int = len(GridSize) ** 2
initialPollution: float = np.random.normal(loc=50, scale=5, size=[1, K])    

# Computing the Phi function values
W: np.ndarray = np.zeros((N, K, K))
for k in range(N):
    for pair in product(list(Location.keys()), repeat=2):
        i, j = pair
        W[k, i, j] = 0 if i == j else phiFunction(
            phi=Phi,
            rho=Rho,
            distance=distanceCalc(coordinateA=Location[i], coordinateB=Location[j], unit=Distance),
            n=timeInterval,
            velocity=windSpeed[k],
            angle=relativeAngle(coordinateA=Location[i], coordinateB=Location[j], windDir=windDirection[k]),
            lag=Lag
        )
    row_norms = np.linalg.norm(W[k, :, :], axis=1)
    row_norms[row_norms == 0] = 1
    W[k, :, :] = W[k, :, :] / row_norms[:, np.newaxis]

In [94]:
# Random initial pollution level
Y: np.ndarray = np.zeros([N, K])
Y[0, :] = initialPollution
for i in range(1, N):
    Y[i, :] =  meanPol + W[i, :, :] @ Y[i-1, :].transpose()

In [95]:
Y

array([[5.09651712e+01, 4.74550078e+01, 4.31999449e+01, 5.09347127e+01,
        4.40442293e+01, 4.80245087e+01, 5.31426491e+01, 4.09414909e+01,
        4.75379555e+01],
       [1.57661026e+02, 1.58933773e+02, 1.60347140e+02, 1.57694777e+02,
        1.60233930e+02, 1.58728532e+02, 1.56946342e+02, 1.61279715e+02,
        1.58835832e+02],
       [4.74501439e+02, 4.73984740e+02, 4.73477705e+02, 4.74581276e+02,
        4.72637014e+02, 4.74005898e+02, 4.73884970e+02, 4.73232234e+02,
        4.73134323e+02],
       [1.36404510e+03, 1.36425630e+03, 1.36442621e+03, 1.36402270e+03,
        1.36423283e+03, 1.36426927e+03, 1.36387033e+03, 1.36456232e+03,
        1.36415643e+03],
       [3.88329067e+03, 3.88322809e+03, 3.88315215e+03, 3.88331996e+03,
        3.88087162e+03, 3.88322493e+03, 3.88189836e+03, 3.88304601e+03,
        3.88179258e+03],
       [1.09604316e+04, 1.09866983e+04, 1.09604696e+04, 1.09855002e+04,
        1.10046564e+04, 1.09855212e+04, 1.09594266e+04, 1.09853902e+04,
        1.0